### 🧩 Imports

Import all the necessary libraries:
- `json` for reading/writing JSON files.
- `pandas` and `numpy` for data manipulation.
- `random` for generating random synthetic data.
- `IsolationForest` from `sklearn` for anomaly detection.
- `joblib` for saving the trained model.

In [1]:
import json
import pandas as pd
from sklearn.ensemble import IsolationForest
import numpy as np
import random
import joblib

### 📊 Generate Dataset

Create synthetic "normal" network traffic data to train an anomaly detection model. Each sample includes:

- `src_port`: randomly selected from common service ports.
- `dst_port`: a random high port number.
- `packet_size`: typical packet sizes.
- `duration_ms`: duration of the communication.
- `protocol`: randomly selected between TCP and UDP.

This data is saved to `training_data.json` for future use.

In [2]:
COMMON_PORTS = [80, 443, 22, 8080]

def generate_normal_data():
    return {
        "src_port": random.choice(COMMON_PORTS),
        "dst_port": random.randint(1024, 65535),
        "packet_size": random.randint(100, 1500),
        "duration_ms": random.randint(50, 500),
        "protocol": random.choice(["TCP", "UDP"])
    }

dataset = [generate_normal_data() for _ in range(1000)]

for d in dataset:
    d["port_diff"] = abs(d["src_port"] - d["dst_port"])
    d["bytes_per_ms"] = round(d["packet_size"] / d["duration_ms"], 3)

open("dataset/training_data.json", "w").close()

with open("dataset/training_data.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [3]:
with open("./dataset/training_data.json") as f:
    raw_data = json.load(f)

df = pd.DataFrame(raw_data)
display(df)

,src_port,dst_port,packet_size,duration_ms,protocol,port_diff,bytes_per_ms
0,22,40222,130,321,TCP,40200,0.405
1,443,39725,938,61,TCP,39282,15.377
2,8080,55274,216,388,UDP,47194,0.557
3,80,19889,260,246,UDP,19809,1.057
4,8080,9112,983,169,UDP,1032,5.817
...,...,...,...,...,...,...,...
995,443,10097,1172,276,UDP,9654,4.246
996,443,54537,1177,373,UDP,54094,3.155
997,80,25374,506,165,UDP,25294,3.067
998,80,9727,442,192,UDP,9647,2.302


### 🧼 Preprocessing Function

Machine learning models like Isolation Forest require **numerical input only**. Any categorical variables, such as the `protocol` column (`TCP`, `UDP`), must be converted into numbers.

We handle this with **one-hot encoding**, using `pd.get_dummies`.

#### 🛠️ Preprocessing Steps:

1. **Identify categorical columns**:
   - In our case, the `protocol` column is categorical (`TCP`, `UDP`).

2. **Use `pd.get_dummies`**:
   - This creates a new binary column for each category.
   - For instance:
     ```
     protocol
     ---------
     TCP   →   protocol_UDP = 0
     UDP   →   protocol_UDP = 1
     ```
   - Setting `drop_first=True` prevents multicollinearity by dropping the first category (`TCP` here), as it can be inferred from the others.

3. **Return a DataFrame with all numerical values**:
   - This is ready for model input.

> ✅ This preprocessing is essential to avoid errors during training and ensure the model can learn from categorical variables.

In [4]:
def preprocess_data(df):
    df_encoded = pd.get_dummies(df, columns=["protocol"], drop_first=True)
    return df_encoded

### 🤖 Train Isolation Forest

The `IsolationForest` algorithm is an unsupervised model used to detect anomalies. It isolates observations by randomly selecting features and splitting values.

- `n_estimators=100`: number of trees in the forest.
- `contamination=0.01`: assumes 1% of the data is anomalous.
- `random_state=42`: ensures reproducibility.

The model is trained on the preprocessed numerical dataset.

In [5]:
processed_data = preprocess_data(df)
model = IsolationForest(contamination=0.1, random_state=42)
model.fit(processed_data)

IsolationForest(contamination=0.1, random_state=42)

### 💾 Save Trained Model

Save the trained model using `joblib`, which allows for efficient serialization and deserialization. This saved model can be reused later for inference or deployment.


In [6]:
joblib.dump(model, "anomaly_model.joblib")

['anomaly_model.joblib']

# predict data

In [7]:
predictions = model.predict(processed_data)
print(np.unique(predictions, return_counts=True))

(array([-1,  1]), array([100, 900]))
